Functional programming, decorator, decorator pattern
====================================================

:warning: This is the hardest training, so buckle up!

:hourglass: 3h

**Outline**:
1. Functional programming and the declarative paradigm
2. Closure and partial
3. Callable classes
4. Decorator (usage, syntactic sugar syntax, creating decorator, best practices (including when to/when not to))
5. The Decorator pattern
6. Closing words

## 1. Functional programming and the declarative paradigm

:hourglass: 20 min

Functional programming belongs to the *declarative* paradigm. Contrary to procedural and OOP, which belong to the same paradigm (imperative), functional programming can feel very different.

Remember that
> A (programming) **paradigmn** is a way to think about, approach and solve a problem. It defines the (conceptual) primitives in which to think in order to create the solution.

In functional programming, the primitives are functions and recursion.

Consider the example below:

In [ ]:
# Procedural
def sum_of_n_plus_1_first_naturals(n):
    s = 0
    for i in range(n+1):
        s += i
    return s

sum_of_n_plus_1_first_naturals(15)

In [ ]:
# Functional
def sum_of_n_plus_1_first_naturals(n):
    if n <= 1:
        return n
    return n + sum_of_n_plus_1_first_naturals(n-1)

sum_of_n_plus_1_first_naturals(15)

In the first example, we explicitly state how the state evolves. In the second, we only state how things relate to one another.

As is evident from the example, Python supports both approaches. 

Note that "nothing" changes in the functional approach: the relation must always be true. That is why, the core of *declarative* programming is all about immutability. The functional part means we express relationships through the use of functions (and recursion). An important concept to be flexible in expressing relationships through functions is the notion of **first-class functions**.


First-class functions mean that you can pass functions to other functions and you can return functions. For instance:

In [ ]:
def filter(ls, f):
    return [x for x in ls if f(x)]

def even_or_odd(return_even):
    def is_even(x):
        return x % 2 == 0
    
    if return_even:
        return is_even
    return lambda x: not is_even(x)


filter(range(10), even_or_odd(return_even=False))

Most often, you want to return a function that is further parametrized. This is done through the **closure** mechanism.

> Of course, there is so much more to talk about in the context of functional programming (accumulator, continuation, functor, monads, inclusion/exclusion principle, etc.) but this is outside of the scope of this training.

## 2. Closure and partial

:hourglass: 20 min

> A **closure** of a function is the set of information that the function captures from the enclosing scopes.

Consider the following example:

In [ ]:
a = 8
def foo(x):
    return a + x

foo(2)

In the example above, `a` is not within the body of `foo`, yet the function is able to access it.

Closures are notably useful for nested functions:

In [ ]:
def create_is_divisor_of(n):
    def is_divisor(x):
        return x % n == 0

    return is_divisor

print(create_is_divisor_of(3))

filter(range(10), create_is_divisor_of(3))

:question: Why not use a function `is_divisor_of(x, n)`?

`is_divisor_of(x, n)` does not follow the interface of the `filter` function. Adapting functions in this way is so frequent, that it is provided out-of-the-box:

In [ ]:
from functools import partial

def is_divisor_of(x, n):
    return x % n == 0

filter(range(10), partial(is_divisor_of, n=3))

> Adaptating to interfaces is not something restricted to functional programming. In OOP, there is the *adaptater*  design pattern for this.

> :skull: Some programming languages (eg. Haskell) naturally treat functions of several variables as parametric families of functions and perform automatic *currying**. In such languages, you can write `is_divisor_of(x)` and this would return a function which can take `n` as input.

> :skull: Did you know that, with closure, you can implement datastructures like pairs, lists/tuples, trees, graphs, etc. as well as manage state in functional programming (which can lead ultimatly to writing operating systems); closure is a very powerful mechanism.

## 3. Callable classes

:hourglass: 10 min

Can we do the same things with classes (well, there is a dedicated section, so you guessed it)?


In [ ]:
class DvisiorPredicate:
    def __init__(self, n: int) -> None:
        self._divisor = n

    def __call__(self, x: int) -> bool:
        return x % self._divisor == 0

filter(range(10), DvisiorPredicate(3))

Turning a class into a function (a callable) is done via the `__call__` dunder.

In such a small example, the overhead of using an object is evident. However, as things gets more complicated, using object will often make the code easier to write (and read!).

### Exercise

:hourglass: 15 min

Write a `twice` function which takes as input a function $f(\cdot)$ and returns the function $f(f(\cdot))$

## 4. Decorator 

Decorating a function means it is modified in some way, with the assumption that the purpose of the function is unchanged. This is useful in many scenarios:
- caching;
- logging and telemetry (monitoring, profiling);
- managing access control;
- validating inputs;
- concurrency;
- etc.

:bulb: decorators are an elegant way to add capabilities with minimal impact on the code and re-usability. 

### Usage
:hourglass: 15 min

The following illustrates how to use a built-in cache of Python:


In [ ]:
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

fibonacci(40)

In [ ]:
import functools

@functools.cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

fibonacci(40)

> :skull: caching (or memoization) brings down the complexity of the first implemention from $O(2^n)$  to $O(n)$. Whereas the former implementation would require more than the age of the universe to compute `fibonacci(100)`, it is instantaneous for the latter.

To be able to create custom decorators, it is necessary to understand that the annotation (`@`) is just a syntactic sugar. It is strictly equivalent to the following:

In [ ]:
import functools

def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# print(fibonacci)

fibonacci = functools.cache(fibonacci)

# print(fibonacci)
fibonacci(40)

So the decorator is simply a function that takes another function as input and returns a function (sounds a lot like first-order functions). The annotation syntax is just a convenient way to express the decoration.

### Creating decorators


:hourglass: 10 min

Now that we know how it works, we can create decorators!

In [ ]:
from typing import Callable, TypeVar, Any
from time import perf_counter, sleep

T = TypeVar("T")

def timeit(f: Callable[..., T]) -> Callable[..., T]:

    def timed_f(*args: Any, **kwargs: Any) -> T:
        start = perf_counter()
        x = f(*args, **kwargs)
        end = perf_counter()
        print(f"Duration of {f.__name__}: {end-start:.2f}")
        return x
    
    return timed_f


@timeit
def long_computation(duration):
    sleep(duration)
    return 42

long_computation(15)

### Exercise

:hourglass: 30 min

Create a decorator to log the start and end of a function. It should mention the timestamp for each. You can use `dt.datetime.now()` for the latter.

### Parametric decorator

:hourglass: 15 min

It is possible to customize the decoration as well, although the syntax becomes complex quickly:

In [ ]:
from typing import Any, Callable, TypeVar
import io
import contextlib

T = TypeVar("T")

def capture_stdout(buffer: io.StringIO) -> Callable[[Callable[..., Any]], Callable[..., Any]]:
    def capturing_closure(f: Callable[..., T]) -> Callable[..., T]:
        def captured_f(*args: Any, **kwargs: Any) -> T:
            with contextlib.redirect_stdout(buffer):
                return f(*args, **kwargs)
        
        return captured_f
    
    return capturing_closure


buffer = io.StringIO()

@capture_stdout(buffer)
def blabla():
    print("This is a called to print during the blabla function")

print("Before blabla")
blabla()
print("After blabla")
print("Buffer:", buffer.getvalue())

Clearly, the code is becoming more complex (and the typing as well). One way out is to use a callable object for the intermediate part:

In [ ]:
from typing import Any, Callable, TypeVar, Generic
import io
import contextlib

T = TypeVar("T")

class Capture(Generic[T]):
    def __init__(self, buffer: io.StringIO, f: Callable[..., T]) -> None:
        self._buffer = buffer
        self._f = f

    def __call__(self, *args: Any, **kwargs: Any) -> T:
        with contextlib.redirect_stdout(self._buffer):
            return self._f(*args, **kwargs)
    

def capture_stdout(buffer: io.StringIO) -> Callable[..., Capture]:
    def capturing_closure(f: Callable[..., T]) -> Callable[..., T]:
        return Capture(buffer, f)
    return capturing_closure


buffer = io.StringIO()


@capture_stdout(buffer)
def blabla():
    print("This is a called to print during the blabla function")

print("Before blabla")
blabla()
print("After blabla")
print("Buffer:", buffer.getvalue())

Note that any (none, all) of the functions can be replaced by a callable object. See what fits best the situation.

Create a retry decorator. It takes as input the number of time it should retry a function. You can use the `random` package to simulate failure.

:coffee: 15 min

###  Best practices 

:hourglass: 10 min

Decoration is a powerful mechanism, rendered elegant thanks to the annotation syntax. It brings encapsulation and re-usability at the cost of additional complexity. Weigh whether the benefits outbalance the added complexity. There are a few situations where you should refrain from using decorators, however:
- a decorator should never change the semantics of the function (the signature should be the same, what the function is basically doing should be unaltered);
- heavy/complex computations hidden in decorators are unexpected;
- mixing concerns: 1 decorator = 1 purpose (you can use several decorators on top of another).


A good practice for decorators is to stamp the decorated function with the information of the original one. Consider the following snippets:

In [ ]:
from time import perf_counter, sleep


def timeit(f):

    def timed_f(*args, **kwargs):
        start = perf_counter()
        x = f(*args, **kwargs)
        end = perf_counter()
        print(f"Duration of {f.__name__}: {end-start:.2f}")
        return x
    
    return timed_f


@timeit
def long_computation(duration: int) -> int:
    """
    This is a long computation

    Parameters
    ----------
    duration: int
        The duration of the long computation (in seconds)

    Return
    ------
    x: int
        The answer to the Ultimate Question of Life, the Universe, and Everything
    """

    sleep(duration)
    return 42

help(long_computation)

In [ ]:
from time import perf_counter, sleep
import functools

def timeit(f):
    @functools.wraps(f)  # simply use the functools wrapper
    def timed_f(*args, **kwargs):
        start = perf_counter()
        x = f(*args, **kwargs)
        end = perf_counter()
        print(f"Duration of {f.__name__}: {end-start:.2f}")
        return x
    
    return timed_f


@timeit
def long_computation(duration: int) -> int:
    """
    This is a long computation

    Parameters
    ----------
    duration: int
        The duration of the long computation (in seconds)

    Return
    ------
    x: int
        The answer to the Ultimate Question of Life, the Universe, and Everything
    """

    sleep(duration)
    return 42

help(long_computation)

## 5. The Decorator pattern

### Exposition

:hourglass: 15 min

Beyond functions, decoration is also an OOP design pattern.


In [ ]:
from abc import ABCMeta, abstractmethod
import pandas as pd
from time import perf_counter

class DataLoader(metaclass=ABCMeta):
    @abstractmethod
    def load(self, database: str, table: str) -> pd.DataFrame:
        raise NotImplementedError()
    

class MockDataLoader(DataLoader):
    def load(self, database: str, table: str) -> pd.DataFrame:
        # Mock data
        return pd.DataFrame(
            data={
                "first_name": ["Bruce", "Clark", "Peter"],
                "last_name": ["Wayne", "Ken", "Parker"],
                "super_hero": ["Batman", "Superman", "Spiderman"],
            }
        )


class LogDataLoarder(DataLoader):
    def __init__(self, decorated: DataLoader) -> None:
        self._decorated = decorated

    def load(self, database: str, table: str) -> pd.DataFrame:
        print(f"loading form table '{table}' ({database})")
        return self._decorated.load(database, table)
    

class TimeDataLoader(DataLoader):
    def __init__(self, decorated: DataLoader) -> None:
        self._decorated = decorated

    def load(self, database: str, table: str) -> pd.DataFrame:
        start = perf_counter()
        x = self._decorated.load(database, table)
        end = perf_counter()
        print(f"Duration: {end-start:.2f}")
        return x
    

data_loader = TimeDataLoader(
    LogDataLoarder(
        MockDataLoader()
    )
)

data_loader.load("my_db", "my_table")


### Exercise

:hourglass: 15 min

Add a class cache decorator for the example above. You can log whether there is a cache hit or miss.

In [ ]:
class Cache(DataLoader):
    pass
    

data_loader = TimeDataLoader(
    LogDataLoarder(
        Cache(
            MockDataLoader(),
        )
    )
)




data_loader.load("my_db", "my_table")
print("Second call")
data_loader.load("my_db", "my_table")


See how the order of decoration changes what happens.

### Notes
:hourglass: 10 min

As we can see from the examples, there is no restriction to work with callables. The decorator is extremely powerful but has a few important caveats:
- in case of long nesting, debugging is hard (a clear repr might help!);
- the decoration hides the base object: we can no longer access it easily;
- the decorator must re-implement all the methods of the base class, which adds a lot of code (mostly simply delegating to the decorated object).

One workaround for the last point is to reserve this design pattern for classes with only one main function. 

:skull: Another through the use of `__getattr__`. The code might not be as readable, however.

In [ ]:
class A:
    def m1(self):
        print("m1 in A")

    def m2(self):
        print("m2 in A")

class B:
    def __init__(self, decorated) -> None:
        self._decorated = decorated

    def m1(self):
        print("m1 in B")

    def __getattr__(self, name: str) -> Any:
        return getattr(self._decorated, name)
    

__o = B(A())
__o.m1()
__o.m2()

## 6. Closing words

:hourglass: 10 min

In this tutorial, we have discussed the functional programming paradigm and what it brings to Python (first-class functions and closure). We then focused on the decorator protocol of Python: how to use decorators, how to make them, and how to parametrize them. 


**Dunderscore**
- `__call__`